<h3>PayPal Transaction Monitoring</h3>

In [1]:
# import libraries
import pandas as pd
import numpy as np
import datetime
from datetime import date, timedelta


In [2]:
# import dataset
dfpreclean = pd.read_csv("data/Paypal_Transactions3.csv")

In [3]:
dfpreclean.head()

,Transaction_ID,Type,Transaction_Type,Customer_Name,Total,Success,Day,Transaction_Notes,Source,Country,Auth_code
0,1234567,Charge,Goods and Services,James,3286,1,1/2/2023,Thanks,Tablet,US,X8JZG7YH
1,9876543,Refund,Friends & Family,Emily,1624,1,1/3/2023,Raffle,Phone,US,D2F3R6KP
2,4567890,Charge,Goods and Services,Liam,2659,1,1/4/2023,Thanks,Desktop,US,Q9L4T1VW
3,7654321,Charge,Goods and Services,Olivia,4897,1,1/5/2023,Thanks,Phone,US,M7N5P0QI
4,2345678,Charge,Friends & Family,Benjamin,3643,1,1/6/2023,Thanks,Desktop,UK,B6K8D3XJ


In [4]:
# Drop unimportant columns
dfpreclean.drop(["Transaction_ID", "Auth_code"], axis=1, inplace=True)

In [5]:
dfpreclean.head(3)

,Type,Transaction_Type,Customer_Name,Total,Success,Day,Transaction_Notes,Source,Country
0,Charge,Goods and Services,James,3286,1,1/2/2023,Thanks,Tablet,US
1,Refund,Friends & Family,Emily,1624,1,1/3/2023,Raffle,Phone,US
2,Charge,Goods and Services,Liam,2659,1,1/4/2023,Thanks,Desktop,US


In [6]:
# check succesfull transaction
dfpreclean2 = dfpreclean[dfpreclean["Success"] == 1]


In [7]:
# Replace NAN values in Transaction_Notes column with N/A
dfpreclean2["Transaction_Notes"].fillna("N/A", inplace=True)

/tmp/ipykernel_44823/2508074308.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfpreclean2["Transaction_Notes"].fillna("N/A", inplace=True)
/tmp/ipykernel_44823/2508074308.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfpreclean2["Transaction_Notes"].fillna("N/A", inplace=True)


In [8]:
dfpreclean2["Day"] = pd.to_datetime(dfpreclean2["Day"])


/tmp/ipykernel_44823/929889925.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfpreclean2["Day"] = pd.to_datetime(dfpreclean2["Day"])


In [9]:
# Check columns
dfpreclean2.columns

Index(['Type', 'Transaction_Type', 'Customer_Name', 'Total', 'Success', 'Day',
       'Transaction_Notes', 'Source', 'Country'],
      dtype='object')

In [10]:
# Change the column order
df = dfpreclean2.loc[:, ['Total', 'Transaction_Type', 'Type', 'Country', 'Source', 'Day',
       'Customer_Name', 'Success', 'Transaction_Notes']]

In [11]:
df.head(3)

,Total,Transaction_Type,Type,Country,Source,Day,Customer_Name,Success,Transaction_Notes
0,3286,Goods and Services,Charge,US,Tablet,2023-01-02,James,1,Thanks
1,1624,Friends & Family,Refund,US,Phone,2023-01-03,Emily,1,Raffle
2,2659,Goods and Services,Charge,US,Desktop,2023-01-04,Liam,1,Thanks


In [12]:
# Sum of all transactions
totalsum = np.sum(df['Total'])
total_transactions = df['Type'].count()

mean_transaction = np.mean(df['Total'])
median_transaction = np.median(df['Total'])
max_transaction = np.max(df['Total'])
#total_unique_customers = df["Customer_Name"].nunique()

In [13]:
chargeonlytransactions = df[df["Type"] == "Charge"]
refundonlytransactions = df[df["Type"] == "Refund"]
chargebackonlytransactions = df[df["Type"] == "Chargeback"]


In [14]:
days90 = pd.to_datetime(date.today() - timedelta(days=90))
days180 = pd.to_datetime(date.today() - timedelta(days=180))


In [15]:
chargetotal = np.sum(chargeonlytransactions["Total"])
charge90days = np.sum(chargeonlytransactions[chargeonlytransactions["Day"] > days90]["Total"])

charge180days = np.sum(chargeonlytransactions[chargeonlytransactions["Day"] > days180]["Total"])


In [16]:
refundtotal = np.sum(refundonlytransactions["Total"])
refund90days = np.sum(refundonlytransactions[refundonlytransactions["Day"] > days90]["Total"])

refund180days = np.sum(refundonlytransactions[refundonlytransactions["Day"] > days180]["Total"])


In [17]:
chargebacktotal = np.sum(chargebackonlytransactions["Total"])
chargeback90days = np.sum(chargebackonlytransactions[chargebackonlytransactions["Day"] > days90]["Total"])

chargeback180days = np.sum(chargebackonlytransactions[chargebackonlytransactions["Day"] > days180]["Total"])


In [18]:
# Check refund rate tifetime
refundratelifetime = (refundtotal/chargetotal)
refundrate90days = (refund90days/charge90days)
refundrate180days = (refund180days/charge180days)

/tmp/ipykernel_44823/2261324578.py:3: RuntimeWarning: invalid value encountered in scalar divide
  refundrate90days = (refund90days/charge90days)
/tmp/ipykernel_44823/2261324578.py:4: RuntimeWarning: invalid value encountered in scalar divide
  refundrate180days = (refund180days/charge180days)


In [19]:
chargebackratelifetime = (chargebacktotal/chargetotal)
chargebackrate90days = (chargeback90days/charge90days)
chargebackrate180days = (chargeback180days/charge180days)

/tmp/ipykernel_44823/1815949388.py:2: RuntimeWarning: invalid value encountered in scalar divide
  chargebackrate90days = (chargeback90days/charge90days)
/tmp/ipykernel_44823/1815949388.py:3: RuntimeWarning: invalid value encountered in scalar divide
  chargebackrate180days = (chargeback180days/charge180days)


In [20]:
pivottablenames = pd.pivot_table(df, index=['Customer_Name'], aggfunc={'Total': np.sum, 'Customer_Name': 'count',})
pivottablenames = pivottablenames.rename(columns={"Customer_Name": "count_of_total", "Total": "sum_of_total"})
pivottablenames = pivottablenames.loc[:,['sum_of_total', 'count_of_total']]
total_unique_customers = pivottablenames['sum_of_total'].count()


/tmp/ipykernel_44823/1134923467.py:1: FutureWarning: The provided callable <function sum at 0x7fe0d84471a0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivottablenames = pd.pivot_table(df, index=['Customer_Name'], aggfunc={'Total': np.sum, 'Customer_Name': 'count',})


In [21]:
avg_transactions_count_per_customer = np.mean(pivottablenames['count_of_total'])
avg_transactions_sum_per_customer = np.mean(pivottablenames['sum_of_total'])

total_unique_customers = pivottablenames["sum_of_total"].count()

In [22]:
pivottabltransactiontype = pd.pivot_table(df, index=['Transaction_Type'], aggfunc={'Transaction_Type': 'count', 'Total': np.sum})
pivottabltransactiontype['totalpercent'] = (pivottabltransactiontype['Total']/totalsum).apply('{:.2%}'.format)
pivottabltransactioncountry = pd.pivot_table(df, index=['Country'], aggfunc={'Country': 'count', 'Total': np.sum})
pivottabltransactioncountry['totalpercent'] = (pivottabltransactioncountry['Total']/totalsum).apply('{:.2%}'.format)


/tmp/ipykernel_44823/637751637.py:1: FutureWarning: The provided callable <function sum at 0x7fe0d84471a0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivottabltransactiontype = pd.pivot_table(df, index=['Transaction_Type'], aggfunc={'Transaction_Type': 'count', 'Total': np.sum})
/tmp/ipykernel_44823/637751637.py:3: FutureWarning: The provided callable <function sum at 0x7fe0d84471a0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivottabltransactioncountry = pd.pivot_table(df, index=['Country'], aggfunc={'Country': 'count', 'Total': np.sum})


In [23]:
pivottabltransactioncountry

,Country,Total,totalpercent
Country,,,
AU,1,3792,0.62%
UK,8,27967,4.57%
US,186,579744,94.81%


In [24]:
firstname = "Patrick"
namefinal = df[df['Customer_Name'].str.contains(firstname, case=False)]

In [25]:
namefinal

,Total,Transaction_Type,Type,Country,Source,Day,Customer_Name,Success,Transaction_Notes


In [26]:
payment_note = df[df['Transaction_Notes'].isna() == False]
flagged_words = 'raffle|razz|lottery'
payment_note_final = payment_note[payment_note['Transaction_Notes'].str.contains(flagged_words, case=False)]


In [27]:
highticketval = 300
highticket = df[df['Total'] >= highticketval].copy()
highticket = highticket.sort_values(by='Total', ascending=False)
highticket

,Total,Transaction_Type,Type,Country,Source,Day,Customer_Name,Success,Transaction_Notes
101,4996,Goods and Services,Charge,US,Desktop,2023-05-01,Benjamin,1,Thank you
122,4993,Goods and Services,Charge,US,Tablet,2023-05-17,Chloe,1,Thanks
144,4990,Goods and Services,Charge,US,Desktop,2023-06-08,Elizabeth,1,Thank you
116,4971,Friends & Family,Charge,US,Desktop,2023-05-16,Daniel,1,N/A
41,4966,Friends & Family,Refund,US,Phone,2023-02-16,Emily,1,Razz
...,...,...,...,...,...,...,...,...,...
75,1105,Goods and Services,Charge,US,Phone,2023-03-27,Noah,1,Thank you
34,1085,Goods and Services,Charge,US,Desktop,2023-02-06,Joseph,1,Thanks
96,1055,Goods and Services,Refund,US,Desktop,2023-04-26,Sofia,1,Thank you
13,1047,Friends & Family,Refund,US,Phone,2023-01-18,Harper,1,Thanks


In [28]:
dup = df.copy()

dup['Customer_Name_next'] = dup['Customer_Name'].shift(1)
dup['Customer_Name_prev'] = dup['Customer_Name'].shift(-1)

dup['created_at_day'] = dup['Day']
dup['created_at_dayprev'] = dup['Day'].shift(-1)
dup['created_at_daynext'] = dup['Day'].shift(1)

dup3 = dup.query('(created_at_day == created_at_dayprev | created_at_day == created_at_daynext) & (Customer_Name == Customer_Name_next | Customer_Name == Customer_Name_prev)')


In [29]:
dfcalc = pd.DataFrame({'totalsum':[totalsum],
                           'mean_transaction':[mean_transaction],
                           'median_transaction':[median_transaction], 
                           'max_transaction':[max_transaction],
                           'total_transactions':[total_transactions],
                           'chargetotal':[chargetotal],
                           'charge90days':[charge90days],
                           'charge180days':[charge180days],
                           'refundtotal':[refundtotal],
                           'refund90days':[refund90days],
                           'refund180days':[refund180days],
                           'refundratelifetime':[refundratelifetime],
                           'refundrate90days':[refundrate90days],
                           'refundrate180days':[refundrate180days],
                           'chargebacktotal':[chargebacktotal],
                           'chargeback90days':[chargeback90days],
                           'chargeback180days':[chargeback180days],
                           'chargebackratelifetime':[chargebackratelifetime],
                           'chargebackrate90days':[chargebackrate90days],
                           'chargebackrate180days':[chargebackrate180days],
                           #'total_unique_customers':[total_unique_customers],                      
                           'avg_transactions_count_per_customer':[avg_transactions_count_per_customer],
                           'avg_transactions_sum_per_customer':[avg_transactions_sum_per_customer],
                           '90 Days':[days90],
                           '180 Days':[days180],
                        })

In [30]:
format_mapping = {"totalsum": '${:,.2f}',
                  "mean_transaction": '${:,.2f}',
                  "median_transaction": '${:,.2f}',
                  "max_transaction": '${:,.2f}',
                  "total_transactions": '{:,.0f}', 
                  'chargetotal': '${:,.2f}',
                  'charge90days': '${:,.2f}',
                  'charge180days': '${:,.2f}',
                  'refundtotal': '${:,.2f}',
                  'refund90days': '${:,.2f}',
                  'refund180days': '${:,.2f}',
                  'refundratelifetime':'{:.2%}',
                  'refundrate90days':'{:.2%}',
                  'refundrate180days':'{:.2%}',
                  'chargebacktotal':'${:,.2f}',
                  'chargeback90days':'${:,.2f}',
                  'chargeback180days':'${:,.2f}',
                  'chargebackratelifetime':'{:.2%}',
                  'chargebackrate90days':'{:.2%}',
                  'chargebackrate180days':'{:.2%}',
                 # "total_unique_customers": '{:,.0f}',
                  "avg_transactions_count_per_customer": '{:,.2f}',
                  "avg_transactions_sum_per_customer": '${:,.2f}',                  
                    }


In [31]:
for key, value in format_mapping.items():
            dfcalc[key] = dfcalc[key].apply(value.format)
